In [1]:
import sympy as sp

sp.init_printing()

In [2]:
t = sp.symbols("t")
X = sp.Function("X")
Y = sp.Function("Y")
Z = sp.Function("Z")

# Define camera properties
csi = sp.symbols("xi")
psi = sp.symbols("psi") 

# Auxiliary variables
alpha = sp.symbols("alpha")
rho = sp.sqrt(X(t) ** 2 + Y(t) ** 2 + Z(t) ** 2)
r = sp.symbols("rho")

# Define xs and y
x = X(t)/(Z(t) + csi * rho)
y = -Y(t)/(Z(t) + csi * rho)

# Define Ji
Ji_0 = sp.diff(x, X(t))
Ji_1 = sp.diff(x, Y(t))
Ji_2 = sp.diff(x, Z(t))
Ji_3 = sp.diff(y, X(t))
Ji_4 = sp.diff(y, Y(t))
Ji_5 = sp.diff(y, Z(t))

Ji = sp.Matrix([[Ji_0, Ji_1, Ji_2],
               [Ji_3, Ji_4, Ji_5]])
Ji_s = Ji * (rho * (Z(t) + csi * rho)**2 )

In [3]:
if False:
    test_1 = Ji_s[0, 0] - (rho * Z(t) + csi * (Y(t)**2 + Z(t)**2))
    test_2 = Ji_s[1, 1] + (rho * Z(t) + csi * (X(t)**2 + Z(t)**2))
    sp.simplify(test_2)

In [4]:
Jm = sp.Matrix([[0, -Z(t), Y(t), -1, 0, 0], 
                [Z(t), 0, -X(t), 0, -1, 0], 
                [-Y(t), X(t), 0, 0, 0, -1]])
Jg = Ji @ Jm

In [5]:
# Try symbolic simplification of x and y given their values
xs = sp.symbols("x")
ys = sp.symbols("y")
alpha = sp.symbols("alpha")
r = sp.symbols("rho")

# Substitute for expressions
Jg_s = Jg.subs({x: xs, y: ys, rho: r})
Jg_s = sp.simplify(Jg_s)
Jg_n = Jg_s.subs({x: xs, y: ys, rho: r})
sp.simplify(Jg_n)

⎡                                 2      2                                    
⎢                            - ρ⋅x ⋅ξ - x ⋅Z(t) - Z(t)                    Y(t)
⎢           -x⋅y             ─────────────────────────                 ───────
⎢                                    ρ⋅ξ + Z(t)                        ρ⋅ξ + Z
⎢                                                                             
⎢             2       2                                      2      ⎛       2 
⎢-ρ⋅ξ⋅Z(t) - Y (t) - Z (t)                              x⋅ξ⋅Y (t) + ⎝ρ - ξ⋅y ⋅
⎢──────────────────────────            -x⋅y             ──────────────────────
⎢ 2  2                 2                                             ρ⋅(ρ⋅ξ + 
⎣ρ ⋅ξ  + 2⋅ρ⋅ξ⋅Z(t) + Z (t)                                                   

                          2                                                   
                    -ρ + x ⋅ξ⋅(ρ⋅ξ + Z(t))        x⋅ξ⋅Y(t)           x⋅(ρ + ξ⋅
───                 ──────────────────────     ────

In [6]:
Jg_numeric = sp.simplify(Jg.subs({X(t): 1, Y(t): 2, Z(t): -3, csi: 0.0})).evalf()

In [7]:
# Implement ground truth expression from Barreto's paper and subtract it to my solution: result needs to be zero
ups = sp.symbols("Upsilon")

# Matrix entries
l11 = xs * ys
l12 = ((1 + xs**2) * ups - ys**2 * csi) / (ups + csi)
l13 = ys
l14 = (1 + xs**2 * (1 - csi * (ups + csi)) + ys**2)/(r*(ups + csi))
l15 = xs*ys*csi/r
l16 = -xs*ups/r

l21 = ((1 + ys**2) * ups - xs**2 * csi) / (ups + csi)
l22 = xs * ys
l23 = -xs
l24 = -xs*ys*csi/r
l25 = (1 + xs**2 + ys**2 * (1 - csi *(ups + csi)))/(r*(ups + csi))
l26 = -ys*ups/r

barreto_L = sp.Matrix([[l11, l12, l13, l14, l15, l16],
                       [l21, l22, l23, l24, l25, l26]])

barreto_L

⎡                     ⎛ 2    ⎞      2       2                     2           
⎢                   Υ⋅⎝x  + 1⎠ - ξ⋅y       x ⋅(-ξ⋅(Υ + ξ) + 1) + y  + 1       
⎢       x⋅y         ─────────────────  y   ────────────────────────────       
⎢                         Υ + ξ                     ρ⋅(Υ + ξ)                 
⎢                                                                             
⎢  ⎛ 2    ⎞    2                                                          2   
⎢Υ⋅⎝y  + 1⎠ - x ⋅ξ                                   -x⋅ξ⋅y              x  + 
⎢─────────────────         x⋅y         -x            ───────             ─────
⎣      Υ + ξ                                            ρ                     

                              ⎤
      x⋅ξ⋅y              -Υ⋅x ⎥
      ─────              ─────⎥
        ρ                  ρ  ⎥
                              ⎥
 2                            ⎥
y ⋅(-ξ⋅(Υ + ξ) + 1) + 1  -Υ⋅y ⎥
───────────────────────  ─────⎥
    ρ⋅(Υ + ξ)              ρ  ⎦

In [8]:
# Substitute with numerics
xs_value = x.subs({X(t): 1, Y(t): 2, Z(t): -3, csi: 0.0})
ys_value = y.subs({X(t): 1, Y(t): 2, Z(t): -3, csi: 0.0})
alp = sp.sqrt(1 + (x**2 + y**2) * (1 - csi**2))
ups_value = alp.subs({X(t): 1, Y(t): 2, Z(t): -3, csi: 0.0})
r_value = rho.subs({X(t): 1, Y(t): 2, Z(t): -3, csi: 0.0})
BL_num = barreto_L.subs({xs: xs_value, ys: ys_value, ups: ups_value, r: r_value, csi: 0.0})

In [9]:
BL_num.evalf()

⎡-0.222222222222222   1.11111111111111   0.666666666666667  0.333333333333333 
⎢                                                                             
⎣ 1.44444444444444   -0.222222222222222  0.333333333333333          0         

         0          0.111111111111111 ⎤
                                      ⎥
 0.333333333333333  -0.222222222222222⎦

In [11]:
-Jg_numeric

⎡-0.222222222222222   1.11111111111111   0.666666666666667  -0.333333333333333
⎢                                                                             
⎣ 1.44444444444444   -0.222222222222222  0.333333333333333          0         

          0          -0.111111111111111⎤
                                       ⎥
  0.333333333333333  0.222222222222222 ⎦